# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [41]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

import time
from time import time
from tqdm import tqdm


from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,recall_score,precision_score

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV 
from sklearn.pipeline import make_pipeline

import xgboost as xgb

import catboost
from catboost import Pool,CatBoost
from catboost import CatBoostClassifier
from catboost.utils import eval_metric
from catboost.utils import get_fpr_curve
from catboost.utils import get_fnr_curve
from catboost.utils import select_threshold
from catboost.utils import get_roc_curve
from catboost.utils import get_confusion_matrix


from pymystem3 import Mystem
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import re

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [42]:
try:
    data = pd.read_csv('/datasets/toxic_comments.csv')
    
except:
    data = pd.read_csv('C:/Users/анатолий/Documents/datasets/toxic_comments.csv')

In [43]:
data

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159566,""":::::And for the second time of asking, when ...",0
159567,You should be ashamed of yourself \n\nThat is ...,0
159568,"Spitzer \n\nUmm, theres no actual article for ...",0
159569,And it looks like it was actually you who put ...,0


In [44]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')


l = WordNetLemmatizer()
def lemmatize_text(text):
    text = text.lower()
    
    text = nltk.word_tokenize(text)
    
    
    lemm_text = ' '.join([l.lemmatize(t) for t in text])  
    cleared_text = re.sub(r'[^a-zA-Z0-9]', ' ',lemm_text )
     
    
    return " ".join(cleared_text.split())


[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [45]:

from tqdm import tqdm
tqdm.pandas()
data['text'] = data['text'].progress_apply(lemmatize_text)

100%|██████████| 159571/159571 [02:36<00:00, 1018.01it/s]


In [46]:
data.duplicated().sum()

799

In [47]:
data = data.drop_duplicates()
data.duplicated().sum()

0

In [48]:
import nltk
from nltk.corpus import stopwords as nltk_stopwords
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Данные лематизированы,стоп-слова исключены,дубликаты удалены.
Приступим к созданию признака и цели.

In [49]:
features = data.drop('toxic',axis=1)

target = data['toxic']


In [50]:
features_train, features_test, target_train, target_test = train_test_split(features,
                                                                            target,
                                                                            test_size=0.3,stratify = target,
                                                                            random_state=5)

In [51]:
features_train = count_tf_idf.fit_transform(features_train['text'].values.astype('U'))
features_test = count_tf_idf.transform(features_test['text'].values.astype('U'))
print(features_train.shape)
print(features_test.shape)

(111140, 140719)
(47632, 140719)


In [52]:
print(target_train.shape)
print(target_test.shape)

(111140,)
(47632,)


С размерностями все хорошо - приступим к созданию и испытанию моделей

## Обучение

In [53]:
#         Подберем гиперпараметры для LogisticRegression модели
ldr = LogisticRegression(random_state=5)

param_grid ={'C': [1.0,10.0],\
             'penalty':['l1','l2']}
grid = GridSearchCV(ldr, param_grid,cv=3,n_jobs=1,scoring='f1')
grid=grid.fit(features_train,target_train)
print('Лучшая мера f1 модели на обучающей выборке:{:.2f}'.format(grid.best_score_))

print('Лучший параметр регуляризации и пенальти на обучающей выборке:',grid.best_params_)

Лучшая мера f1 модели на обучающей выборке:0.75
Лучший параметр регуляризации и пенальти на обучающей выборке: {'C': 10.0, 'penalty': 'l2'}


In [54]:
t0 = time()
clf = LogisticRegression(**grid.best_params_)
clf.fit(features_train, target_train)
pred = clf.predict(features_test)
t_lgr = time()-t0
f1_lgr = f1_score(target_test,pred)
print('Время обучения и предсказания модели, сек :',t_lgr)
print('Мера f1 для модели LogisticRegression :',f1_score(target_test,pred).round(2))

Время обучения и предсказания модели, сек : 50.12242817878723
Мера f1 для модели LogisticRegression : 0.77


In [55]:
#  рассмотрим модель CatBoostClassifier
booster = CatBoostClassifier(iterations=150,learning_rate = 1.0, verbose=False, eval_metric= 'F1')
t0=time()
booster.fit(features_train, target_train,
            eval_set=(features_test, target_test),
            
            plot=True);
 


test_preds = booster.predict(features_test)
train_preds = booster.predict(features_train)
t_cat = time() - t0
f1_cat = f1_score(y_true=target_test,y_pred=test_preds)
print('Время обучения и предсказания модели, сек :',t_cat)
print('Метрика f1 на обучающей выборке :', f1_score(y_true=target_train,y_pred=train_preds))
print('Метрика f1 на тестовой выборке :', f1_score(y_true=target_test,y_pred=test_preds))



MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Время обучения и предсказания модели, сек : 374.6681447029114
Метрика f1 на обучающей выборке : 0.8376385519242663
Метрика f1 на тестовой выборке : 0.7545237282348924


In [56]:
xgb_clf = xgb.XGBClassifier()
t0=time()
xgb_clf.fit(features_train, target_train, eval_set=[(features_test, target_test)], eval_metric="auc" , verbose=False)

pred_test = xgb_clf.predict(features_test)
pred_train = xgb_clf.predict(features_train)
t_xgb = time() - t0
f1_xgb = f1_score(y_true=target_test,y_pred=pred_test)

print('Время обучения и предсказания модели, сек :',t_xgb)
print('Метрика  f1 на тестовой', f1_score(y_true=target_test,y_pred=pred_test))
print('Метрика f1 на обучающей', f1_score(y_true=target_train,y_pred=pred_train))
 

Время обучения и предсказания модели, сек : 257.013534784317
Метрика  f1 на тестовой 0.7216443163303672
Метрика f1 на обучающей 0.7766228256257955


In [57]:
d = {'model': ['LogisticRegression','CatBoost','XGBoost'],\
     'f1_score':[f1_lgr,f1_cat,f1_xgb], \
     'fit_predict_time': [t_lgr,t_cat,t_xgb]}
    

display( pd.DataFrame(data=d))

,model,f1_score,fit_predict_time
0,LogisticRegression,0.769070,50.122428
1,CatBoost,0.754524,374.668145
2,XGBoost,0.721644,257.013535


## Выводы

После подготовки данных были испытаны три модели машинного обучения.
И по времени и по результату вычисления метрики f1 лучший результат показала модель LogisticRegression.
Наверное так и должно было быть ведь вектор признака это 0 и 1.А далее сигмоидальная функция активации очень точно разделяет классы.
Метрика выше 0.75 получена - задача проекта выполнена.